# 前言

学习趋势检验。

# 为什么需要趋势检验

In [43]:
# 一组不相关的数值
x <- seq(1, 10, 0.1)
y <- sin(x)^(cos(x) + 1)

cor.test(x, y, method = "pearson")


	Pearson's product-moment correlation

data:  x and y
t = -1.165, df = 52, p-value = 0.2493
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.4097558  0.1130961
sample estimates:
       cor 
-0.1594944 


# 回归方法实现趋势检验

In [60]:
# 读取SPSS数据
library(foreign)

data <- read.spss("./data/reg4trend.sav", to.data.frame = TRUE)

str(data)

'data.frame':	54 obs. of  11 variables:
 $ X....: num  1 2 3 4 5 6 7 8 9 10 ...
 $ x1   : num  3 2 2 2 3 3 2 3 2 1 ...
 $ x2   : num  1 0 1 0 0 0 0 0 0 0 ...
 $ x3   : num  0 1 0 0 0 1 1 1 0 0 ...
 $ x4   : num  1 1 1 1 1 1 0 1 0 1 ...
 $ x5   : num  0 0 0 0 0 0 0 1 0 0 ...
 $ x6   : num  0 0 0 0 1 0 0 0 0 0 ...
 $ x7   : num  1 1 1 1 1 2 1 1 1 1 ...
 $ x8   : num  1 0 0 0 1 1 0 0 1 0 ...
 $ y    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ PGR_1: num  1 0 0 0 1 1 0 0 0 0 ...
 - attr(*, "variable.labels")= Named chr [1:11] "" "" "" "" ...
  ..- attr(*, "names")= chr [1:11] "\320\362\272\305" "x1" "x2" "x3" ...


In [63]:
fit <- glm(y ~ x1 + x2, data = data, family = binomial())

library(broom)
tidy(fit)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-2.2233200,1.0325911,-2.153147,0.03130716
x1,0.7115257,0.4233223,1.680813,0.09279918
x2,1.0800713,0.6249976,1.728121,0.08396659


In [66]:
library(tidyverse)
distinct(data, x1)

x1
<dbl>
3
2
1
4


In [68]:
data <- data %>% 
  mutate(
    x1_2 = ifelse(x1 == 2, 1, 0),
    x1_3 = ifelse(x1 == 3, 1, 0),
    x1_4 = ifelse(x1 == 4, 1, 0),
  )

fit <- glm(y ~ x1_2 + x1_3 + x1_4, data = data, family = binomial())

summary(fit)


Call:
glm(formula = y ~ x1_2 + x1_3 + x1_4, family = binomial(), data = data)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)  
(Intercept)   -1.609      1.095  -1.469   0.1418  
x1_2           1.204      1.169   1.030   0.3031  
x1_3           2.148      1.194   1.799   0.0720 .
x1_4           2.708      1.592   1.701   0.0889 .
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 74.786  on 53  degrees of freedom
Residual deviance: 68.564  on 50  degrees of freedom
AIC: 76.564

Number of Fisher Scoring iterations: 4


# Cochran-Armitage趋势检验

In [74]:
install.packages("DescTools")


The downloaded binary packages are in
	/var/folders/fv/c0dh9y6s7638rh5fdr3lgcmr0000gn/T//Rtmp5DHWJq/downloaded_packages


In [71]:
# Create a contingency table with English names
data <- matrix(
  c(20, 10,   # Low dose: 20 non-responders, 10 responders
    15, 25,   # Medium dose: 15 non-responders, 25 responders
    5, 30),   # High dose: 5 non-responders, 30 responders
  nrow = 3, 
  byrow = TRUE,
  dimnames = list(
    Dose = c("Low", "Medium", "High"),
    Response = c("No", "Yes")
  )
)

# Print the table
print(data)

        Response
Dose     No Yes
  Low    20  10
  Medium 15  25
  High    5  30


In [75]:
library(DescTools)

# Run the test
result <- CochranArmitageTest(data)

# View results
print(result)


	Cochran-Armitage test for trend

data:  data
Z = -4.3257, dim = 3, p-value = 1.521e-05
alternative hypothesis: two.sided

